###Apartado 3: Transferencia de modelos y ajuste fino con ResNet50 para la base de datos Caltech-UCSD.

In [0]:
# Alumno : Daniel Bolaños Martínez 76592621E
# Asignatura : Visión por Computador
# Práctica 2 : Introducción a Keras para la clasificación de imágenes

# -*- coding: utf-8 -*-

#########################################################################
################### OBTENER LA BASE DE DATOS ############################
#########################################################################

# Descargar las imágenes de http://www.vision.caltech.edu/visipedia/CUB-200.html
# Descomprimir el fichero.
# Descargar también el fichero list.tar.gz, descomprimirlo y guardar los ficheros
# test.txt y train.txt dentro de la carpeta de imágenes anterior. Estos 
# dos ficheros contienen la partición en train y test del conjunto de datos.

from google.colab import drive
drive.mount('/content/drive')
! unzip /content/drive/My\ Drive/VC/images.zip -d /content/

In [0]:
#########################################################################
################ CARGAR LAS LIBRERÍAS NECESARIAS ########################
#########################################################################

# Terminar de rellenar este bloque con lo que vaya haciendo falta

# Importar librerías necesarias
import numpy as np
import keras
import keras.utils as np_utils
from keras.preprocessing.image import load_img, img_to_array
import matplotlib.pyplot as plt

# Importar el optimizador a usar
from keras.optimizers import SGD

# Importar DataGenerator
from keras.preprocessing.image import ImageDataGenerator

# Importar modelos y capas específicas que se van a usar

from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras import backend as K
# Import Early Stopping
from keras.callbacks import EarlyStopping

# Importar el modelo ResNet50 y su respectiva función de preprocesamiento,
# que es necesario pasarle a las imágenes para usar este modelo

from keras.applications.resnet import ResNet50, preprocess_input

# Importar el optimizador a usar
from keras.optimizers import SGD

In [0]:

#########################################################################
################## FUNCIÓN PARA LEER LAS IMÁGENES #######################
#########################################################################

# Dado un fichero train.txt o test.txt y el path donde se encuentran los
# ficheros y las imágenes, esta función lee las imágenes
# especificadas en ese fichero y devuelve las imágenes en un vector y 
# sus clases en otro.

def leerImagenes(vec_imagenes, path):
  clases = np.array([img.split('/')[0] for img in vec_imagenes])
  imagenes = np.array([img_to_array(load_img(path + "/" + img, 
                                             target_size = (224, 224))) 
                       for img in vec_imagenes])
  return imagenes, clases

#########################################################################
############# FUNCIÓN PARA CARGAR EL CONJUNTO DE DATOS ##################
#########################################################################

# Usando la función anterior, y dado el path donde se encuentran las
# imágenes y los archivos "train.txt" y "test.txt", devuelve las 
# imágenes y las clases de train y test para usarlas con keras
# directamente.

def cargarDatos(path):
  # Cargamos los ficheros
  train_images = np.loadtxt(path + "/train.txt", dtype = str)
  test_images = np.loadtxt(path + "/test.txt", dtype = str)
  
  # Leemos las imágenes con la función anterior
  train, train_clases = leerImagenes(train_images, path)
  test, test_clases = leerImagenes(test_images, path)
  
  # Pasamos los vectores de las clases a matrices 
  # Para ello, primero pasamos las clases a números enteros
  clases_posibles = np.unique(np.copy(train_clases))
  for i in range(len(clases_posibles)):
    train_clases[train_clases == clases_posibles[i]] = i
    test_clases[test_clases == clases_posibles[i]] = i

  # Después, usamos la función to_categorical()
  train_clases = np_utils.to_categorical(train_clases, 200)
  test_clases = np_utils.to_categorical(test_clases, 200)
  
  # Barajar los datos
  train_perm = np.random.permutation(len(train))
  train = train[train_perm]
  train_clases = train_clases[train_perm]

  test_perm = np.random.permutation(len(test))
  test = test[test_perm]
  test_clases = test_clases[test_perm]
  
  return train, train_clases, test, test_clases

#########################################################################
######## FUNCIÓN PARA OBTENER EL ACCURACY DEL CONJUNTO DE TEST ##########
#########################################################################

# Esta función devuelve el accuracy de un modelo, definido como el 
# porcentaje de etiquetas bien predichas frente al total de etiquetas.
# Como parámetros es necesario pasarle el vector de etiquetas verdaderas
# y el vector de etiquetas predichas, en el formato de keras (matrices
# donde cada etiqueta ocupa una fila, con un 1 en la posición de la clase
# a la que pertenece y 0 en las demás).

def calcularAccuracy(labels, preds):
  labels = np.argmax(labels, axis = 1)
  preds = np.argmax(preds, axis = 1)
  
  accuracy = sum(labels == preds)/len(labels)
  
  return accuracy

#########################################################################
## FUNCIÓN PARA PINTAR LA PÉRDIDA Y EL ACCURACY EN TRAIN Y VALIDACIÓN ###
#########################################################################

# Esta función pinta dos gráficas, una con la evolución de la función
# de pérdida en el conjunto de train y en el de validación, y otra
# con la evolución del accuracy en el conjunto de train y en el de
# validación. Es necesario pasarle como parámetro el historial
# del entrenamiento del modelo (lo que devuelven las funciones
# fit() y fit_generator()).

def mostrarEvolucion(hist):

  loss = hist.history['loss']
  val_loss = hist.history['val_loss']
  plt.plot(loss)
  plt.plot(val_loss)
  plt.legend(['Training loss', 'Validation loss'])
  plt.show()

  acc = hist.history['acc']
  val_acc = hist.history['val_acc']
  plt.plot(acc)
  plt.plot(val_acc)
  plt.legend(['Training accuracy', 'Validation accuracy'])
  plt.show()

In [0]:
# input image dimensions
num_classes = 200
batch_size = 32
epochs = 20

# cargamos los datos
x_train, y_train, x_test, y_test = cargarDatos('/content/images')

## Usar ResNet50 preentrenada en ImageNet como un extractor de características

# Definir un objeto de la clase ImageDataGenerator para train y otro para test
# con sus respectivos argumentos.

datagen_train = ImageDataGenerator(preprocessing_function=preprocess_input)
datagen_test = ImageDataGenerator(preprocessing_function=preprocess_input)

train = datagen_train.flow(x_train, y_train, batch_size = 1, shuffle = False)
test = datagen_test.flow(x_test, batch_size = 1, shuffle = False)

# Definir el modelo ResNet50 (preentrenado en ImageNet y sin la última capa).

resnet50 = ResNet50(include_top=False, weights='imagenet', pooling='avg')
resnet50.summary()

# Extraer las características las imágenes con el modelo anterior.

train_features = resnet50.predict_generator(train)
test_features = resnet50.predict_generator(test)

In [0]:
# Las características extraídas en el paso anterior van a ser la entrada
# de un pequeño modelo de dos capas Fully Conected, donde la última será la que 
# nos clasifique las clases de Caltech-UCSD (200 clases). De esta forma, es 
# como si hubiéramos fijado todos los parámetros de ResNet50 y estuviésemos
# entrenando únicamente las capas añadidas. Definir dicho modelo.

input_shape = (2048,)

model = Sequential()
model.add(Dense(512, activation='relu', input_shape=input_shape))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))
model.summary()

# explicar por qué selecciono ese optimizador

opt = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=opt,
              metrics=['acc'])

# Una vez tenemos el modelo base, y antes de entrenar, vamos a guardar los
# pesos aleatorios con los que empieza la red, para poder reestablecerlos
# después y comparar resultados entre no usar mejoras y sí usarlas.

weights = model.get_weights()

In [0]:
model.set_weights(weights)

epochs = 40

# En la función fit() puedes usar el argumento validation_split

histograma = model.fit(train_features, y_train, epochs=epochs, validation_split=0.1,
                       callbacks = [EarlyStopping(monitor = 'val_acc', patience = 4, restore_best_weights = True)])
mostrarEvolucion(histograma)

preds = model.predict(test_features)
score = calcularAccuracy(y_test, preds)
print("Predicción sobre conjunto Test")
print("Test accuracy = " + str(score))
# Predicción sobre el conjunto de Train
preds = model.predict(train_features)
score = calcularAccuracy(y_train, preds)
print("Predicción sobre conjunto Train")
print("Train accuracy = " + str(score))

In [0]:
## Reentrenar ResNet50 (fine tunning)

# Definir un objeto de la clase ImageDataGenerator para train y otro para test
# con sus respectivos argumentos.

datagen_train_norm = ImageDataGenerator(preprocessing_function=preprocess_input, featurewise_center = True, 
                                        featurewise_std_normalization = True, horizontal_flip=True, zoom_range=0.0, validation_split=0.1)
datagen_test_norm = ImageDataGenerator(preprocessing_function=preprocess_input, featurewise_center = True, 
                                       featurewise_std_normalization = True)

# Añadir nuevas capas al final de ResNet50 (recuerda que es una instancia de
# la clase Model).

epochs=20

x = resnet50.output
x = Dense(512, activation='relu')(x)
x = Dropout(0.75)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
last = Dense(num_classes, activation='softmax')(x)
new_model = Model(inputs = resnet50.input, outputs = last)

opt = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

new_model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=opt,
              metrics=['acc'])

# Una vez tenemos el modelo base, y antes de entrenar, vamos a guardar los
# pesos aleatorios con los que empieza la red, para poder reestablecerlos
# después y comparar resultados entre no usar mejoras y sí usarlas.

weights = new_model.get_weights()

In [0]:
new_model.set_weights(weights)

datagen_train_norm.fit(x_train)
datagen_test_norm.fit(x_train)
train_norm = datagen_train_norm.flow(x_train, y_train, batch_size = batch_size, subset = 'training')
validation_norm = datagen_train_norm.flow(x_train, y_train, batch_size = batch_size, subset = 'validation')

histograma = new_model.fit_generator(train_norm, steps_per_epoch = len(x_train)*0.9/batch_size, epochs = epochs, 
                    validation_data = validation_norm, validation_steps = len(x_train)*0.1/batch_size)

mostrarEvolucion(histograma)
preds = new_model.predict_generator(datagen_test_norm.flow(x_test, batch_size = 1, shuffle = False), steps=len(x_test))
score = calcularAccuracy(y_test, preds)
print("Predicción sobre conjunto Test")
print("Test accuracy = " + str(score))
# Predicción sobre el conjunto de Train
preds = new_model.predict_generator(datagen_train_norm.flow(x_train, batch_size = 1, shuffle = False), steps = len(x_train))
score = calcularAccuracy(y_train, preds)
print("Predicción sobre conjunto Train")
print("Train accuracy = " + str(score))